<a href="https://colab.research.google.com/github/nickwan/nwds-stream-notebooks/blob/main/24_nwvs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# font size stuff (doesn't help with nwvs, just notebook output text size for stream)

In [ ]:
def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '24px'
      break
    }
  }
  '''))
get_ipython().events.register('pre_run_cell', increase_font)

RULES
# What is nwvs?
Also known as Nick Wan Versus  
I challenge all competitive data scientists to a 2-hour data science modeling prediction competition on Kaggle. Everyone (including me) will not know what the dataset is until 9pm ET. From 9pm ET to 11pm ET, we have to develop a model and submit model predictions to a Kaggle competition page (which will be revealed at 9pm ET).  

Last year's nwvs can be found here:  
https://www.kaggle.com/competitions/portland-housing-age-prediction-nwvs-s00e01  

Everyone who beats me on the leaderboard and posts their notebook on Kaggle can receive a free gifted sub from me. For those who beat me and don't provide a notebook of their work, I will anonymously gift the chat the sub. For example: if I place 6th, I owe 6 subs. If the top 3 people don't post their notebooks on Kaggle, I will gift 3 of the 6 subs to chat randomly.  

NWVS starts at 9pm ET on Tuesday, 1/16.  

# MY CODE dejj

to do list:  
~~start big timer first  
get link  
nightbot update  
tweet  
download data  
set the path  
speed run (pred)  
execute rest of plan  ~~
4 legit entries before the end of the first hour (pred? ask chat)   
place 1st on public board one time in the first 15 mins of the 2nd hour (pred)   
start gamba for top 5 with 15 mins left, maybe 10 left (pred)  

my plan:  
1. regular pipeline
2. lightgbm deny?  
3. tune xgb  
4. tune cb  
5a. if denied, swap for h2o small  
5b. if accepted, tune lgbm  
6. h2o big  

In [ ]:
!pip install catboost
!pip install optuna
!pip install h2o

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.2/265.2 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.44.0.3-py2.py3-none-any.whl size=265293968 sha256=3f32f19100c3f26087d9f962a3a1b9663aa97654c6411ab6323b374b781443fe
  Stored in directory: /root/.cache/pip/wheels/77/9a/1c/2da26f943fd46b57f3c20b54847b936b9152b831dc7447cf71
Successfully built h2o


In [ ]:
import os
import optuna as opt
import pandas as pd
import numpy as np
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, GroupKFold
from sklearn.neural_network import MLPRegressor
import h2o
h2o.init()

from h2o.estimators.deeplearning import H2ODeepLearningEstimator

<IPython.core.display.Javascript object>

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpkh21fetn
  JVM stdout: /tmp/tmpkh21fetn/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpkh21fetn/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 5 days
H2O_cluster_name:,H2O_from_python_unknownUser_1vt6oe
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train['test'] = 0
test['test'] = 1
data = pd.concat([train, test], ignore_index=True)
cols = [
    'race', 'date', 'time', 'circuit', 'driver', 'constructor'
]

for col in cols:
  data[f'{col}_code'] = data[col].astype('category').cat.codes

<IPython.core.display.Javascript object>

In [ ]:
x = data.dtypes
x[x!='object'].index

<IPython.core.display.Javascript object>

Index(['id', 'latitude', 'longitude', 'altitude', 'carNumber',
       'avgDriverFinish', 'avgConstructorFinish', 'lapNumber', 'lapPosition',
       'pitStop', 'pitCount', 'pitTime_ms', 'lapTime_ms', 'test', 'race_code',
       'date_code', 'time_code', 'circuit_code', 'driver_code',
       'constructor_code'],
      dtype='object')

In [ ]:
sd_not_lead_lap.head()

<IPython.core.display.Javascript object>

NameError: name 'sd_not_lead_lap' is not defined

In [ ]:
avg_lead_lap = data.loc[data['lapPosition']<=5, ['driver','lapTime_ms']].groupby(['driver'], as_index=False).mean()
sd_lead_lap = data.loc[data['lapPosition']<=5, ['driver','lapTime_ms']].groupby(['driver'], as_index=False).std()
avg_pit = data.loc[:, ['constructor','pitTime_ms']].groupby(['constructor'], as_index=False).mean()
sd_pit = data.loc[:, ['constructor','pitTime_ms']].groupby(['constructor'], as_index=False).std()

avg_track_lap = data.loc[data['lapPosition']<=10, ['circuit_code', 'lapNumber','lapTime_ms']].groupby(['circuit_code', 'lapNumber'], as_index=False).mean()
sd_track_lap = data.loc[data['lapPosition']<=10, ['circuit_code', 'lapNumber','lapTime_ms']].groupby(['circuit_code', 'lapNumber'], as_index=False).std()

avg_not_lead_lap = data.loc[data['lapPosition'].between(5,10), ['driver','lapTime_ms']].groupby(['driver'], as_index=False).mean()
sd_not_lead_lap = data.loc[data['lapPosition'].between(5,10), ['driver','lapTime_ms']].groupby(['driver'], as_index=False).std()

data = data.merge(avg_track_lap.rename(columns={'lapTime_ms':'avgTrackLap'}), how='left').fillna(-1)
data = data.merge(sd_track_lap.rename(columns={'lapTime_ms':'sdTrackLap'}), how='left').fillna(-1)
data = data.merge(avg_lead_lap.rename(columns={'pitTime_ms':'avgLeadLap'}), how='left').fillna(-1)
data = data.merge(sd_lead_lap.rename(columns={'pitTime_ms':'sdLeadLap'}), how='left').fillna(-1)
data = data.merge(avg_pit.rename(columns={'pitTime_ms':'avgPitTime'}), how='left').fillna(-1)
data = data.merge(sd_pit.rename(columns={'pitTime_ms':'sdPitTime'}), how='left').fillna(-1)

data = data.merge(avg_not_lead_lap.rename(columns={'lapTime_ms':'avgNotLeadLap'}), how='left').fillna(-1)
data = data.merge(sd_not_lead_lap.rename(columns={'lapTime_ms':'sdNotLeadLap'}), how='left').fillna(-1)
data.shape

<IPython.core.display.Javascript object>

(50714, 32)

In [ ]:
data.columns.values

<IPython.core.display.Javascript object>

array(['id', 'race', 'date', 'time', 'circuit', 'latitude', 'longitude',
       'altitude', 'driver', 'carNumber', 'constructor',
       'avgDriverFinish', 'avgConstructorFinish', 'lapNumber',
       'lapPosition', 'pitStop', 'pitCount', 'pitTime_ms', 'lapTime_ms',
       'test', 'race_code', 'date_code', 'time_code', 'circuit_code',
       'driver_code', 'constructor_code', 'avgTrackLap', 'sdTrackLap',
       'avgPitTime', 'sdPitTime', 'avgNotLeadLap', 'sdNotLeadLap'],
      dtype=object)

In [ ]:
data.loc[(data['race']=='British Grand Prix') & (data['driver']=='Ricciardo, Daniel'), ['id','date','driver','lapNumber','pitCount', 'lapTime_ms']].sort_values(['date','lapNumber']).head(50)

<IPython.core.display.Javascript object>

,id,date,driver,lapNumber,pitCount,lapTime_ms
47524,47525,2015-07-05,"Ricciardo, Daniel",1,0,-1.0
48536,48537,2015-07-05,"Ricciardo, Daniel",2,0,-1.0
9452,9453,2015-07-05,"Ricciardo, Daniel",3,0,140362.0
49425,49426,2015-07-05,"Ricciardo, Daniel",4,0,-1.0
13305,13306,2015-07-05,"Ricciardo, Daniel",5,0,102341.0
49554,49555,2015-07-05,"Ricciardo, Daniel",6,0,-1.0
44419,44420,2015-07-05,"Ricciardo, Daniel",7,0,-1.0
17421,17422,2015-07-05,"Ricciardo, Daniel",8,0,100499.0
14224,14225,2015-07-05,"Ricciardo, Daniel",9,0,101008.0
31468,31469,2015-07-05,"Ricciardo, Daniel",10,0,-1.0


In [ ]:
cat_cols = [
    'time_code', 'circuit_code',
       'driver_code', 'constructor_code','pitStop', 'pitCount',

]
feats = [
    'latitude', 'longitude', 'altitude',
       'avgDriverFinish', 'avgConstructorFinish', 'lapNumber', 'lapPosition',
       'pitStop', 'pitCount', 'pitTime_ms',
       'circuit_code', 'driver_code',
       'constructor_code', 'avgTrackLap', 'sdTrackLap',
       'avgPitTime', 'sdPitTime', 'avgNotLeadLap', 'sdNotLeadLap'
]
target = 'lapTime_ms'
model_data = data.loc[data['test']==0].dropna(subset=feats+[target])
model_data.shape

<IPython.core.display.Javascript object>

(30000, 32)

In [ ]:
def tune(trial):
  p = {
      'objective':'reg:squarederror',
      'num_round':trial.suggest_int('num_round', 100,500),
      'eta':trial.suggest_float('eta', 0, 0.3),
      'reg_lambda':trial.suggest_float('reg_lambda', 0, 3),
      'reg_alpha':trial.suggest_float('reg_alpha', 0, 3),
  }
  model = xgb.train(p, xtrain)
  _s = pd.Series(model.predict(xtest), index=test.index)
  loss = mean_squared_error(test[target], _s)
  return loss

<IPython.core.display.Javascript object>

In [ ]:
folds = 3
kf = GroupKFold(folds)
dfp = pd.DataFrame()
for train_idx, test_idx in kf.split(model_data, groups=model_data['circuit']):
  train = model_data.iloc[train_idx]
  test = model_data.iloc[test_idx]
  # test[target] = 0
  xtrain = xgb.DMatrix(train.loc[:, feats], train[target])
  xtest = xgb.DMatrix(test.loc[:, feats], test[target])
  study = opt.create_study(direction='minimize')
  study.optimize(tune, n_trials=20)
  _dfp = pd.DataFrame([study.best_params])
  dfp = pd.concat([dfp,_dfp])

pp = dfp.mean().to_dict()
p = {'objective':'reg:squarederror',}
xgbp = {**p,**pp}
xgbp['num_round'] = int(round(xgbp['num_round']))

<IPython.core.display.Javascript object>

[I 2024-01-26 02:31:58,356] A new study created in memory with name: no-name-74ead017-5715-4c6e-beda-c515c02af6b6
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:31:58] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_round" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-01-26 02:31:58,449] Trial 0 finished with value: 7706563080.359192 and parameters: {'num_round': 296, 'eta': 0.026595807256909087, 'reg_lambda': 1.9327455425798203, 'reg_alpha': 0.8484742117004257}. Best is trial 0 with value: 7706563080.359192.
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:31:58] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_round" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-01-26 02:31:58,501] Trial 1 finished with value: 7274833175.602879 and parameters: {'num_round': 386, 'eta': 0.18423908625287408, 'reg_lambda': 1.7393710908220816, 'reg_alpha': 1.3734050290255087}. Best is trial 1 wi

In [ ]:
def tune(trial):
  p = {
      'objective':'regression',
      'eta':trial.suggest_float('eta', 0, 0.3),
      'n_estimators':trial.suggest_int('n_estimators', 100,300),
      'reg_lambda':trial.suggest_float('reg_lambda', 0, 3),
      'reg_alpha':trial.suggest_float('reg_alpha', 0, 3),
  }
  model = lgb.train(p, ltrain)
  _s = pd.Series(model.predict(test.loc[:, feats]), index=test.index)
  loss = mean_squared_error(test[target], _s)
  return loss

<IPython.core.display.Javascript object>

In [ ]:
dfp = pd.DataFrame()
for train_idx, test_idx in kf.split(model_data, groups=model_data['circuit']):
  train = model_data.iloc[train_idx]
  test = model_data.iloc[test_idx]
  # test[target] = 0
  xtrain = xgb.DMatrix(train.loc[:, feats], train[target])
  xtest = xgb.DMatrix(test.loc[:, feats], test[target])
  ltrain = lgb.Dataset(train.loc[:, feats], train[target])
  ltest = lgb.Dataset(test.loc[:, feats], test[target])
  study = opt.create_study(direction='minimize')
  study.optimize(tune, n_trials=20)
  _dfp = pd.DataFrame([study.best_params])
  dfp = pd.concat([dfp,_dfp])

pp = dfp.mean().to_dict()
p = {'objective':'regression',}
lgbp = {**p,**pp}
lgbp['n_estimators'] = int(round(lgbp['n_estimators']))

<IPython.core.display.Javascript object>

[I 2024-01-26 02:32:02,533] A new study created in memory with name: no-name-83a264bf-98a9-4e41-99fc-2fbb7e278f15
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003873 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1334
[LightGBM] [Info] Number of data points in the train set: 20016, number of used features: 19
[LightGBM] [Info] Start training from score 94561.504396


[I 2024-01-26 02:32:03,260] Trial 0 finished with value: 6270001123.882374 and parameters: {'eta': 0.052430702767424904, 'n_estimators': 192, 'reg_lambda': 1.08666044760867, 'reg_alpha': 1.203352264429674}. Best is trial 0 with value: 6270001123.882374.
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005504 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1334
[LightGBM] [Info] Number of data points in the train set: 20016, number of used features: 19
[LightGBM] [Info] Start training from score 94561.504396


...

[I 2024-01-26 02:32:48,333] Trial 17 finished with value: 70835308.87926015 and parameters: {'eta': 0.16821172999664652, 'n_estimators': 162, 'reg_lambda': 1.164695824944642, 'reg_alpha': 1.9111256536285182}. Best is trial 4 with value: 61572593.01350487.
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004162 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1366
[LightGBM] [Info] Number of data points in the train set: 19984, number of used features: 19
[LightGBM] [Info] Start training from score 97692.863441


[I 2024-01-26 02:32:49,309] Trial 18 finished with value: 64039985.662779056 and parameters: {'eta': 0.013958037258312878, 'n_estimators': 277, 'reg_lambda': 0.02846054150568378, 'reg_alpha': 2.4746067252933677}. Best is trial 4 with value: 61572593.01350487.
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004108 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1366
[LightGBM] [Info] Number of data points in the train set: 19984, number of used features: 19
[LightGBM] [Info] Start training from score 97692.863441


[I 2024-01-26 02:32:49,736] Trial 19 finished with value: 60891136.95603324 and parameters: {'eta': 0.08483650842469129, 'n_estimators': 118, 'reg_lambda': 2.5118298975833757, 'reg_alpha': 0.3838313451059572}. Best is trial 19 with value: 60891136.95603324.


In [ ]:
def tune(trial):
  p = {
      'n_estimators':trial.suggest_int('n_estimators', 100,200),
      'eta':trial.suggest_float('eta', 0, 0.3),
      'reg_lambda':trial.suggest_float('reg_lambda', 0, 3),
  }
  model = cb.CatBoostRegressor(**p, verbose=False)
  model.fit(train.loc[:, feats], train[target])
  _s = pd.Series(model.predict(test.loc[:, feats]), index=test.index)
  loss = mean_squared_error(test[target], _s)
  return loss

<IPython.core.display.Javascript object>

In [ ]:
dfp = pd.DataFrame()
for train_idx, test_idx in kf.split(model_data, groups=model_data['circuit']):
  train = model_data.iloc[train_idx]
  test = model_data.iloc[test_idx]
  # test[target] = 0
  xtrain = xgb.DMatrix(train.loc[:, feats], train[target])
  xtest = xgb.DMatrix(test.loc[:, feats], test[target])
  ltrain = lgb.Dataset(train.loc[:, feats], train[target])
  ltest = lgb.Dataset(test.loc[:, feats], test[target])

  study = opt.create_study(direction='minimize')
  study.optimize(tune, n_trials=10)
  _dfp = pd.DataFrame([study.best_params])
  dfp = pd.concat([dfp,_dfp])

cbp = dfp.mean().to_dict()
cbp['n_estimators'] = int(round(cbp['n_estimators']))
# cbp = {**p,**pp}

<IPython.core.display.Javascript object>

[I 2024-01-26 02:32:49,834] A new study created in memory with name: no-name-fcdd6b2d-f492-4523-82f1-ac5521b7b146
[I 2024-01-26 02:32:50,672] Trial 0 finished with value: 7540700182.184094 and parameters: {'n_estimators': 147, 'eta': 0.1689656205895288, 'reg_lambda': 1.8726964106536799}. Best is trial 0 with value: 7540700182.184094.
[I 2024-01-26 02:32:51,411] Trial 1 finished with value: 7806565529.613244 and parameters: {'n_estimators': 155, 'eta': 0.03371525441832829, 'reg_lambda': 1.9439643083150284}. Best is trial 0 with value: 7540700182.184094.
[I 2024-01-26 02:32:52,353] Trial 2 finished with value: 7391136926.193222 and parameters: {'n_estimators': 186, 'eta': 0.23629557252967168, 'reg_lambda': 1.3978553516118848}. Best is trial 2 with value: 7391136926.193222.
[I 2024-01-26 02:32:53,459] Trial 3 finished with value: 7340905286.281172 and parameters: {'n_estimators': 130, 'eta': 0.20523994669733206, 'reg_lambda': 1.4044000855482337}. Best is trial 3 with value: 7340905286.281

In [ ]:
folds = 5
kf = GroupKFold(folds)
out1 = pd.Series()
out2 = pd.Series()
out3 = pd.Series()
for train_idx, test_idx in kf.split(model_data, groups=model_data['circuit']):
  train = model_data.iloc[train_idx]
  test = model_data.iloc[test_idx]
  # test[target] = 0
  xtrain = xgb.DMatrix(train.loc[:, feats], train[target])
  xtest = xgb.DMatrix(test.loc[:, feats], test[target])
  ltrain = lgb.Dataset(train.loc[:, feats], train[target])
  ltest = lgb.Dataset(test.loc[:, feats], test[target])

  model = xgb.train(xgbp, xtrain)
  _s1 = pd.Series(model.predict(xtest), index=test.index)
  out1 = pd.concat([out1, _s1])

  model = cb.CatBoostRegressor(**cbp)
  model.fit(train.loc[:, feats], train[target])
  _s2 = pd.Series(model.predict(test.loc[:, feats]), index=test.index)
  out2 = pd.concat([out2, _s2])

  model = lgb.train(lgbp, ltrain)
  _s = pd.Series(model.predict(test.loc[:, feats]), index=test.index)
  _s3 = pd.Series(model.predict(test.loc[:, feats]), index=test.index)
  out3 = pd.concat([out3, _s3])


model_data[f'{target}_xgb'] = out1
model_data[f'{target}_cb'] = out2
model_data[f'{target}_lgb'] = out3

<IPython.core.display.Javascript object>

<ipython-input-18-781352b8eaed>:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  out1 = pd.Series()
<ipython-input-18-781352b8eaed>:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  out2 = pd.Series()
<ipython-input-18-781352b8eaed>:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  out3 = pd.Series()
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:33:17] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_round" } are not used.

  warnings.warn(smsg, UserWarning)


0:	learn: 51005.4747101	total: 5.61ms	remaining: 735ms
1:	learn: 46632.5201617	total: 11ms	remaining: 718ms
2:	learn: 42707.1513397	total: 17.1ms	remaining: 735ms
3:	learn: 39193.7951051	total: 25.4ms	remaining: 812ms
4:	learn: 36125.3608856	total: 30.5ms	remaining: 775ms
5:	learn: 33450.1302281	total: 35.2ms	remaining: 739ms
6:	learn: 30902.2976784	total: 40.1ms	remaining: 716ms
7:	learn: 28836.9012686	total: 49.7ms	remaining: 770ms
8:	learn: 26821.4396041	total: 55.1ms	remaining: 753ms
9:	learn: 24942.8330255	total: 60ms	remaining: 732ms
10:	learn: 23306.5057565	total: 65.1ms	remaining: 716ms
11:	learn: 21918.2763257	total: 69.9ms	remaining: 699ms
12:	learn: 20753.3297302	total: 74.7ms	remaining: 684ms
13:	learn: 19691.6720339	total: 79.8ms	remaining: 672ms
14:	learn: 18796.4400026	total: 85ms	remaining: 663ms
15:	learn: 18008.6776690	total: 89.7ms	remaining: 650ms
16:	learn: 17289.8345771	total: 94.3ms	remaining: 638ms
17:	learn: 16624.0999793	total: 99.1ms	remaining: 627ms
18:	lear

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004812 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1415
[LightGBM] [Info] Number of data points in the train set: 23873, number of used features: 19
[LightGBM] [Info] Start training from score 95099.778327


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004971 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1419
[LightGBM] [Info] Number of data points in the train set: 24442, number of used features: 19
[LightGBM] [Info] Start training from score 95189.703134


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:33:23] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_round" } are not used.

  warnings.warn(smsg, UserWarning)


0:	learn: 41265.0749224	total: 5.71ms	remaining: 747ms
1:	learn: 37015.7790312	total: 11.8ms	remaining: 766ms
2:	learn: 33262.1523149	total: 17.8ms	remaining: 766ms
3:	learn: 29912.8411285	total: 23.1ms	remaining: 740ms
4:	learn: 27364.6323943	total: 28.4ms	remaining: 721ms
5:	learn: 24768.5763854	total: 33.9ms	remaining: 713ms
6:	learn: 22401.4146110	total: 39.3ms	remaining: 702ms
7:	learn: 20278.0796355	total: 44.6ms	remaining: 691ms
8:	learn: 18418.9015162	total: 49.9ms	remaining: 682ms
9:	learn: 16789.2393744	total: 55.5ms	remaining: 677ms
10:	learn: 15518.4040039	total: 60.7ms	remaining: 668ms
11:	learn: 14245.8694330	total: 66.3ms	remaining: 663ms
12:	learn: 13156.1040865	total: 71.7ms	remaining: 656ms
13:	learn: 12161.8636785	total: 77.1ms	remaining: 650ms
14:	learn: 11331.9881316	total: 85.8ms	remaining: 669ms
15:	learn: 10601.3077324	total: 91.9ms	remaining: 666ms
16:	learn: 9980.2555350	total: 97.3ms	remaining: 658ms
17:	learn: 9454.6020352	total: 102ms	remaining: 648ms
18:	l

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003336 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1369
[LightGBM] [Info] Number of data points in the train set: 23927, number of used features: 19
[LightGBM] [Info] Start training from score 91081.871902


In [ ]:
[
    mean_squared_error(model_data[target], model_data[f"{target}_xgb"])**0.5,
    mean_squared_error(model_data[target], model_data[f"{target}_cb"])**0.5,
    mean_squared_error(model_data[target], model_data[f"{target}_lgb"])**0.5,
]

<IPython.core.display.Javascript object>

[111078.3688788478, 59803.41629886789, 55128.5507721433]

[65631.05732838159, 64410.03829021576, 70924.77368354828]

In [ ]:
hp = {
    'epochs':10,
    'hidden':[200,200],
}
htrain = h2o.H2OFrame(model_data)
htrain[target] = htrain[target].asfactor()
nn = H2ODeepLearningEstimator(**hp)
nn.train(x=[f'{target}_xgb',f'{target}_cb',f'{target}_lgb'], y=target, training_frame=htrain)

In [ ]:
nn = MLPRegressor()
nn.fit(model_data.loc[:, [f'{target}_xgb',f'{target}_cb',f'{target}_lgb']], model_data[target])

<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor()

In [ ]:
train = data.loc[data['test']==0]
test = data.loc[data['test']==1]
test[target] = 0
xtrain = xgb.DMatrix(train.loc[:, feats], train[target])
xtest = xgb.DMatrix(test.loc[:, feats], test[target])
ltrain = lgb.Dataset(train.loc[:, feats], train[target])
ltest = lgb.Dataset(test.loc[:, feats], test[target])

model = xgb.train(xgbp, xtrain)
test[f'{target}_xgb'] = pd.Series(model.predict(xtest), index=test.index)


model = cb.CatBoostRegressor(**cbp)
model.fit(train.loc[:, feats], train[target])
test[f'{target}_cb'] = pd.Series(model.predict(test.loc[:, feats]), index=test.index)

model = lgb.train(lgbp, ltrain)
test[f'{target}_lgb'] = pd.Series(model.predict(test.loc[:, feats]), index=test.index)

test[target] = pd.Series(nn.predict(test.loc[:, [f'{target}_xgb',f'{target}_cb',f'{target}_lgb']]), index=test.index)
test.loc[:, ['id', target]].to_csv('newlul.csv', index=False)


<IPython.core.display.Javascript object>

<ipython-input-22-6b6acca6aa20>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[target] = 0
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:01:05] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_round" } are not used.

  warnings.warn(smsg, UserWarning)
<ipython-input-22-6b6acca6aa20>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[f'{target}_xgb'] = pd.Series(model.predict(xtest), index=test.index)


0:	learn: 57326.2815049	total: 28.6ms	remaining: 3.75s
1:	learn: 51546.3309292	total: 38.7ms	remaining: 2.51s
2:	learn: 46473.8167704	total: 46.1ms	remaining: 1.98s
3:	learn: 42434.2286398	total: 53.9ms	remaining: 1.72s
4:	learn: 38567.7822739	total: 61.1ms	remaining: 1.55s
5:	learn: 35566.6109478	total: 68.8ms	remaining: 1.45s
6:	learn: 32609.6391151	total: 88.9ms	remaining: 1.59s
7:	learn: 30025.8336357	total: 116ms	remaining: 1.8s
8:	learn: 27781.1489239	total: 136ms	remaining: 1.86s
9:	learn: 25859.3047272	total: 160ms	remaining: 1.95s
10:	learn: 24183.4454462	total: 178ms	remaining: 1.96s
11:	learn: 22867.6211177	total: 193ms	remaining: 1.93s
12:	learn: 21759.5287445	total: 210ms	remaining: 1.93s
13:	learn: 20682.1929885	total: 231ms	remaining: 1.95s
14:	learn: 19776.4950315	total: 238ms	remaining: 1.86s
15:	learn: 18983.6647741	total: 248ms	remaining: 1.79s
16:	learn: 18396.2805319	total: 271ms	remaining: 1.83s
17:	learn: 17643.0232930	total: 280ms	remaining: 1.78s
18:	learn: 171

<ipython-input-22-6b6acca6aa20>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[f'{target}_cb'] = pd.Series(model.predict(test.loc[:, feats]), index=test.index)
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009064 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1486
[LightGBM] [Info] Number of data points in the train set: 30000, number of used features: 19
[LightGBM] [Info] Start training from score 94459.793133


<ipython-input-22-6b6acca6aa20>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[f'{target}_lgb'] = pd.Series(model.predict(test.loc[:, feats]), index=test.index)
<ipython-input-22-6b6acca6aa20>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[target] = pd.Series(nn.predict(test.loc[:, [f'{target}_xgb',f'{target}_cb',f'{target}_lgb']]), index=test.index)
